#Projekt ChatBot

#####Cijeli projekt najbolje je skinuti i pokrenuti preko cmd-a, pokretajući ga kao python main.py.  

#Instaliranje potrebnih paketa

In [ ]:
!pip install nltk
!pip install numpy
!pip install tensorflow
!pip install tflearn

#Čitanje input podataka

###Kao podatke koje ChatBot čita kao input odabrao sam json file strukturu (dictionary) u kojoj može postojati više setova podataka, podaci su raspoređeni po tagovima. Odnosno tagovi reprezentiraju vrste rečenica i sa sobom nose pripadne odgovore koje ChatBot kasnije nasumično odabire.


In [5]:
import json
import pprint

input_data = json.loads(open('data.json').read())

pprint.pprint(input_data)

{'trainset': [{'patterns': ['Hi there.',
                            'Howdy.',
                            'Greetings.',
                            "What's up?",
                            'Morning.',
                            'Afternoon.',
                            'Evening.',
                            "What's going on?",
                            'Hey! There you are.',
                            "How's everything?",
                            'Good to see you.',
                            'Great to see you.',
                            'Nice to see you.',
                            'Hello, nice to meet you.',
                            'Hello, nice to see you.',
                            'Hello there.',
                            'Long time no see.',
                            'Hey, good to see you.',
                            'Hey, is it really you?',
                            'Look who it is!',
                            'Hello again.',
                    

#Parsiranje podataka iz json-a

###Samo isječak koda iz data.py datoteke
###Napočetku samo importamo proizvoljni word2vec model

In [ ]:
def rawDataToTraining(filename):
    new_model = gensim.downloader.load('glove-wiki-gigaword-300')
    # model = Word2Vec.load("word2vec.model").wv
    training = []
    tagresponse = {}
    tags = []
    docy = []

    trainingdata = json.loads(open(f'{filename}.json').read())
    for sets in trainingdata:
        for data in trainingdata[sets]:
            for pat in data['patterns']:
                result = [0] * 300
                for word in simple_preprocess(pat):
                    result = np.add(result, new_model[word])
                
                training.append(result)

                docy.append(data['tag'])

                if data['tag'] not in tags:
                        tags.append(data['tag'])

        for resp in trainingdata[sets]:
                if resp['tag'] in tagresponse.keys():
                    tagresponse[resp['tag']].extend(resp['responses'])
                else:  
                    tagresponse[resp['tag']] = resp['responses']
    
    tags = sorted(tags)
    outputdata = []
    out_empty = [0] * len(tags)

    for i in range(len(training)):
        
        output = out_empty[:]

        output[tags.index(docy[i])] = 1
        outputdata.append(output)
    

    training = np.array(training)
    outputdata = np.array(outputdata)

    
    return training, outputdata, tags, tagresponse

#Word2Vec i AI za treniranje

####Kao input podataka za moj model odabrao sam Word2Vec reprezentaciju jedne rečenice. Svaku riječ u rečenici pretvaramo u vektor veličine 300 i redom zbrajamo vrijednosti za cijelu rečenicu. Takav vektor predajemo modelu da se na njemu uči, a kao output modelu je vektor duljine ukupnog broja različitih tagova koje je smo parsirali ranije i za taj tag je 1 sve ostalo su 0.

####Word2Vec model koji sam odabrao je glove-wiki-gigaword-300 kao gotov natrenirani model koji svaku riječ pretvara u vektor dimenzije 300.

####Možemo i sami trenirati svoj model.
####Isječak koda za treniranje vlastitog modela iz data.py datoteke

In [ ]:
def word2vecCreate():
    dataset = api.load("text8")
    data = [d for d in dataset]
    data_train = data[:2500]
    model = Word2Vec(sentences = data_train, window = 10, min_count = 1, workers = cpu_count(), vector_size = 300) 
    model.save("word2vec.model")

word2vecCreate()

#Kreiranje i treniranje modela

####Za dani model odabrao sam da će input biti veličine [None, training[0]], gdje training[0] predstavlja jednu rečenicu odnosno vektor veličine 300, a None da je sve to jedna dimenzija.


####Model se sastoji od 2 skrivena sloja prvi od 16 potpuno povezanih node-ova i drugi od 8. Aktivacijska funkcija za ovaj model je softmax kako bi podaci bili reprezentirani kao postoci (0.34 = 34%).


####Broj epoha za ovaj model iznosi 500, a batch size 10.

####Isječak koda iz main.py datoteke.


In [ ]:
try:
    with open("data.pickle", "rb") as f:
        training, outputdata, tags, tagresponse = pickle.load(f)
except:
    training, outputdata, tags, tagresponse = rawDataToTraining('data')
    with open("data.pickle", "wb") as f:
        pickle.dump((training, outputdata, tags, tagresponse), f)

#input layer
net = tflearn.input_data(shape = [None, len(training[0])])
#hidden layers
net = tflearn.fully_connected(net, 16)
net = tflearn.fully_connected(net, 8)
#output layers
net = tflearn.fully_connected(net, len(outputdata[0]), activation = 'softmax')
#regresija
net = tflearn.regression(net)
#odabir modela
chatbot = tflearn.DNN(net)

try:
    chatbot.load("ChatBotEN")
except:
# #fitanje podataka modelu
    chatbot.fit(X_inputs = training, Y_targets = outputdata, n_epoch = 500, batch_size = 10, show_metric = True)
    chatbot.save("ChatBotEN")

#Dodavanje dodatnih mogućnosti modelu

####Za svoj model dodao sam neke dodatne funkcionalnosti, a to je da za tag weather chatbot povuče stvarne podatke o vremenu preko pythona i weather API-ja.


#####Isječak koda iz main.py datoteke.


In [ ]:
def Weather():
    apikey = "eaa1ff36c65a8953740ec81b7e9f4666"

    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    complete_url = base_url + "appid=" + apikey + "&q=Osijek"

    response = requests.get(complete_url)
    x = response.json()
    if x["cod"] != "404":

        y = x["main"]
        temp = y["temp"] - 272.15
        
        z = x["weather"]
        desc = z[0]["description"]

        pressure = y["pressure"]
        hum = y['humidity']

    else:
        print(" City Not Found ")
    
    
    return temp, desc, pressure, hum

#Testiranje modela

####Kako bih okvirno znao koliko je moj model precizan kreiran je sličan json file za testiranje i nova funkcija TestMetric, koja uspoređuje odgovor ChatBot-a sa stvarnim odgovorima i bilježi kasnije računa postotak u obliku 0.xx (xx%).

####Na ovom testnom file-u preciznost je 0.8372093023255814 (~83%).

####Isječak koda za testiranje za proizvoljni json file iz main.py datoteke.


In [ ]:
def TestMetric(filename):
    score = 0
    total = 0
    new_model = gensim.downloader.load('glove-wiki-gigaword-300')
    trainingdata = json.loads(open(f'{filename}.json').read())
    for sets in trainingdata:
        for data in trainingdata[sets]:
            for pat in data['patterns']:
                result = [0] * 300
                for word in simple_preprocess(pat):
                    parseword = new_model[word]
                    result = np.add(result, parseword)
            
                result = chatbot.predict([result])[0]
                result_index = numpy.argmax(result)
                tag = tags[result_index]
                total += 1
                if tag == data['tag']:
                    score += 1 

    return score / total

#Razgovor sa ChatBot-om


####ChatBot-a pokrećemo pozivom funkcije Chat koja u sebi ima još neke značajke kao npr. ako neku riječ ne može pretvoriti u vektor preko Word2Vec-a onda će bot reći da ne razumije što je korisnik htio reći. Isključivo zbog toga što Word2Vec kao model vraća error za riječi koje ne može prebaciti.


####Isječak koda za funkcije Chat iz main.py datoteke.

In [ ]:
def Chat():
    new_model = gensim.downloader.load('glove-wiki-gigaword-300')
    print("Start talking with the bot! (For quit just type 'quit')")
    while True:
        inputSentence = input("You: ")
        if inputSentence.lower() == "quit":
            break
        
        try:
            result = [0] * 300
            for word in simple_preprocess(inputSentence):
                    parseword = new_model[word]
                    result = np.add(result, parseword)
            
            result = chatbot.predict([result])[0]
            result_index = numpy.argmax(result)

            tag = tags[result_index]
            if tag == "time":
                dtime = datetime.now()
                print("ChatBot: " + random.choice(tagresponse[tag]) + str(dtime.hour), ':', str(dtime.minute))
            elif tag == "music":
                print("ChatBot: For now I can only write songs name, not sing or play them, here is your song: " + random.choice(tagresponse[tag]))
            elif tag == "weather":
                temp, desc, pressure, hum = Weather()
                print("ChatBot: " + random.choice(tagresponse[tag]) + f"temperature: {round(temp, 2)} ° C, description: {desc}, pressure : {pressure} mb, humidity: {hum}%")
            else:
                print("ChatBot: " + random.choice(tagresponse[tag]))
        except:
                print("ChatBot: Sorry I didn't understand that, try typing something else.")


Chat()